In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import sys

sys.path.append('../abstractive')

from model import get_response, create_prompt

dataset_path = "../../Dataset/multilingua.pk"
with open(dataset_path, 'rb') as f:
  dataset = pickle.load(f)

dataset

,document,summary,source
0,"When learning how to solve a Rubik’s cube, it ...",Solve a cube from many different sides rather ...,English
1,"Die richtige Methode anzuwenden, um ein Baby z...",Merke dir das Akronym BACK. Vermeide den „Haus...,German
2,يتيح كل موقع من مواقع التواصل الاجتماعي خيارات...,احذفه أو الغِ صداقته أو متابعته على مواقع التو...,Arabic
3,如果你的礼物接收者居住在寒冷的地方这将留给他一个持久的回忆并且很实用。如果不知道如何织，可以...,织一条毛绒绒暖和的围巾。 个性化的衣服。 给猫或马缝一件衣服。,Chinese
4,Utiliza una base de cobertura ligera que luzca...,Utiliza base de maquillaje con moderación. Enf...,Spanish
...,...,...,...
152946,A breakdown can result from trauma or death of...,Reflect on any recent loss or trauma. Think ab...,English
152947,Aplícate un tono más intenso y luminoso sobre ...,Elige un color de tinte labial más intenso y l...,Spanish
152948,Het is normaal dat je de eerste paar weken na ...,Vraag je arts om een behandeling voor rokerslo...,Dutch
152949,Cari botol anggur yang memiliki catatan tertul...,Kunjungi toko anggur dan mintalah rekomendasi ...,Indonesian


In [2]:
group_lang = {source: group.reset_index(drop=True) for source, group in dataset.groupby('source')}

for lang in ['Hindi', 'Indonesian', 'Japanese', 'Korean']:
    group_lang.pop(lang, None)

counts = {lang: len(df) for lang, df in group_lang.items()}
counts

{'Arabic': 6047,
 'Chinese': 3714,
 'Czech': 1414,
 'Dutch': 6447,
 'English': 28856,
 'French': 13003,
 'German': 11778,
 'Italian': 10187,
 'Portuguese': 16386,
 'Russian': 10532,
 'Spanish': 22767,
 'Turkish': 950,
 'Vietnamese': 4031}

In [3]:
import evaluate

def compute_rouge_scores(summary, reference):
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=[summary], references=[reference])
    return {
        "rouge1": results["rouge1"],
        "rouge2": results["rouge2"],
        "rougeL": results["rougeL"]
    }
    